<a href="https://colab.research.google.com/github/koleshjr/Fruit_Classifier_Project/blob/main/Understanding_FastApi_with_a_fastai_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FASTAI WITH FASTAPI
All the below has been gotten from the link below

https://towardsai.net/p/l/fastapi%E2%80%8A-%E2%80%8Acreate-and-deploy-hot-dog-detector#respond

### File Structure 
.
├── LICENSE <br>
├── README.md <br>
├── app <br>
│   ├── __init__.py <br>
│   ├── json_models <br>
│   │   ├── __init__.py <br>
│   │   └── request.py <br>
│   ├── main.py <br>
│   ├── models <br>
│   │   ├── __init__.py <br>
│   │   └── model.pkl <br>
│   └── utils.py <br>
├── notebooks <br>
│   └── HotDogOrNot.ipynb <br>
└── requirements.txt <br>

In [ ]:
# run this line from the parent directory to install all the necessary requirements
$ pip3 install -r requirements.txt

### Utils.py

In [ ]:
from typing import Dict
from io import BytesIO
import numpy as np
from PIL import Image
from fastai.vision.all import *

#read the image
def read_image(file: bytes) -> PILImage:
    img = Image.open(BytesIO(file))
    fastimg = PILImage.create(np.array(img.convert('RGB')))

    return fastimg



# prediction function and returns the output in the format you want
def predict_hotdog(image) -> Dict:
    path = Path()
    inference_model = load_learner(path/'models/model.pkl')
    is_hotdog, _ , probs = inference_model.predict(image)
    if probs[1] > 0.5:
        return {
            "prediction": "hotdog",
            "probability": probs[1].item()
        }
    
    return {
        "prediction": "not hotdog",
        "probability": probs[0].item()
    }

### response.py

This file that resides within the json_models directory contains aPydantic class for the response model of our API.

In short, Pydantic is a fantastic library that provides data validations using python type annotations at runtime.

This library is especially useful for building APIs and has superb interoperability with FastAPI.

In [ ]:
from pydantic import BaseModel

class Response(BaseModel):
    prediction: str
    probability: float

### main.py

Finally, this is the file that our application will run in production.

First, we create a FastAPI class object. Then, we decorate our predict function using this class object. @app.post indicates that our request will be a post request followed by the path for our API /predict/ .

We also have a parameter called response_model here which accepts our Pydantic model as input. This parameter is to validate our response at runtime.

Finally, our predict function has a query parameter called myfile of type UploadFile which will be used to read our image file in the API.

This variable is passed into the read_image function we saw before which will convert our image data in bytes to a PILImage object which we can easily use to feed our FastAI model.

Finally, the image is passed into our predict_hotdog function which serves predictions and returns a dictionary object as a response

In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastai.vision.all import *
from utils import read_image, predict_hotdog
from json_models.response import Response

app = FastAPI()

@app.post("/predict/", response_model=Response)
async def predict(myfile: UploadFile = File(...)):
    image = read_image(await myfile.read())

    return predict_hotdog(image)

### Testing
In order to manually test out the API, you have to run it first.

Go to the app directory in your repo and type the following command:

$ uvicorn main:app --reload 

Note: The following excerpt is taken directly from the FastAPI docs.

The command uvicorn main:app refers to:

    main: the file main.py (the Python "module").
    app: the object created inside of main.py with the line app = FastAPI().
    –reload: make the server restart after code changes. Only use for development.

On your terminal, you should see the following line:

INFO: Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit

Open http://127.0.0.1:8000/docs on your browser of choice.

### Dockerize